In [1]:
# Always reload modules to have the current version
%reload_ext autoreload
%autoreload 2


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from ranking.util import json_lines as jl
from ranking.util import dataset_paths as dp
import pandas as pd
import numpy as np


In [3]:
def calculate_tfidf_for_dataset(corpus) -> pd.DataFrame:
    vectorizer = TfidfVectorizer()
    result = vectorizer.fit_transform(corpus)
    idf_df = pd.DataFrame.sparse.from_spmatrix(result, columns=vectorizer.get_feature_names())
    return idf_df

def lookup_tfidf_weight(df_tfidf: pd.DataFrame, storage_id, word):
    if word in df_tfidf.columns:
        return df_tfidf.at[storage_id, word]
    else:
        return 0

def apply_tfidf_weights_to_doc(tfidf: pd.DataFrame, storage_id, doc):
    return [(word, lookup_tfidf_weight(tfidf, storage_id, word)) for word in doc.split()]

def create_tfidf_dataset(df: pd.DataFrame, min_unique_words=5):
    groups = df.groupby('storageId')
    corpus = groups['docContent'].first()
    tfidf = calculate_tfidf_for_dataset(corpus)
    eval_dataset = groups.first()
    eval_dataset['n_unique_words'] = eval_dataset['docContent'].str.split().apply(lambda x: np.unique(x).size)
    eval_dataset = eval_dataset[eval_dataset['n_unique_words'] >= min_unique_words]
    eval_dataset['tfidf'] = eval_dataset.apply(lambda row: apply_tfidf_weights_to_doc(tfidf, row.name, row['docContent']), axis=1)
    eval_dataset['docQuery'] = eval_dataset.apply(lambda row: get_query_from_doc(row, get_n_words_to_extract(row)), axis=1)
    return eval_dataset

def get_n_words_to_extract(row, percent=0.3):
    n_unique_words = row['n_unique_words']
    n = round(n_unique_words * percent)
    return n

def get_query_from_doc(row, n):
    unique_weighed_words = list(dict.fromkeys(row['tfidf']))
    max_n_scored_words = sorted(unique_weighed_words, key=lambda word_weight: word_weight[1], reverse=True)[:n]
    query = ' '.join([word for word, _ in max_n_scored_words])
    return query

In [4]:
dataset = jl.read_dataset(dp.tokenized_unique_functions_corpus)
tfidf_ds = create_tfidf_dataset(dataset)
jl.write_dataset(tfidf_ds[['docType', 'docQuery']], 'complete-tfidf-evalset.jsonl')
